In [1]:
import requests
import base64

CLIENT_ID = '7f1c2c8ab19a4f33baa21ad10410fae5'
CLIENT_SECRET = '631ea95684b24a118ebe263c093120ea'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()


Access token obtained successfully.


In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def trending_playlist_data(pid,access_token):
    sp= spotipy.Spotify(auth=access_token)
    
    playlist_tracks = sp.playlist_tracks(pid, fields='items(track(id, name, artists, album(id, name)))')
    music_data=[]

    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
    
        audio_features= sp.audio_features(track_id)[0] if track_id!='Not available' else None

        try:
            album_info= sp.album(album_id) if album_id!='Not available' else None
            release_date= album_info['release_date'] if album_info else None
        except:
            release_date= None
        
        try:
            track_info= sp.track(track_id) if track_id!='Not available' else None
            popularity= track_info['popularity'] if track_info else None
        except:
            popularity= None

        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)
        
    df= pd.DataFrame(music_data)
    return df
    

In [3]:
pid='37i9dQZF1DXcBWIGoYBM5M'
music_df= trending_playlist_data(pid,access_token=access_token)
music_df[['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID', 'Popularity', 'Release Date', 'Duration (ms)']]

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms)
0,Houdini,Dua Lipa,Houdini,5pTaRVLwZOFObIbRBubmeb,4OMJGnvZfDvsePyCwRGO7X,87,2023-11-09,185918
1,greedy,Tate McRae,greedy,3UOV8XvCwMKaATRNXrYCjN,3rUGC1vUpkDG9CZFHMur1t,100,2023-09-15,131872
2,Water,Tyla,Water,22sXXkKgjEuawIFL1e1tRw,5aIVCx5tnk0ntmdiinnYvw,96,2023-07-28,200256
3,Agora Hills,Doja Cat,Scarlet,6DmPNcfpkXBVRJsEIJY9tl,7dJYggqjKo71KI9sLzqCs8,92,2023-09-22,265360
4,Standing Next to You,Jung Kook,GOLDEN,5pSk3c3wVwnb2arb6ohCPU,2KslE17cAJNHTsI2MI0jb2,96,2023-11-03,206020
5,Strangers,Kenya Grace,Strangers,18ogtNq9F7DmMkNYO6Xb4k,5mjYQaktjmjcMKcUIcqz4s,98,2023-09-01,172965
6,MONACO,Bad Bunny,nadie sabe lo que va a pasar mañana,4FftCsAcXXD1nFO9RFUNFO,4MjDJD8cW7iVeWInc2Bdyj,98,2023-10-13,267194
7,Is It Over Now? (Taylor's Version) (From The V...,Taylor Swift,1989 (Taylor's Version),64LU4c1nfjz1t4VnGhagcg,1Iq8oo9XkmmvCQiGOfORiz,94,2023-10-26,229478
8,Snooze,SZA,SOS,07w0rG5TETcyihsEIZR3qG,4iZ4pt7kvcaH6Yo8UoZ4s2,92,2022-12-09,201800
9,My Love Mine All Mine,Mitski,The Land Is Inhospitable and So Are We,2Cn1d2KgbkAqbZCJ1RzdkA,3vkCueOmm7xQDoJ17W1Pm3,97,2023-09-15,137773


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

data= music_df

Recommending the latest releases to users

In [5]:
# Function to calculate weighted popularity scores based on release date
def calculate_weight_probablity(release_date):
    release_date= datetime.strptime(release_date,'%Y-%m-%d')

    #time span between release date and today's date
    timespan= datetime.now()-release_date

    #more recent releases have higher weight
    weight= 1/ (timespan.days+1)
    return weight

In [6]:
scaler= MinMaxScaler()
music_feature= music_df[['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                        'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_feature_scaled= scaler.fit_transform(music_feature)

In [10]:
def content_based_recommendation(input_song, num_rec=5):
    if input_song not in music_df['Track Name'].values:
        print(input_song,'not in dataset. Please enter a valid song name')
        return
    inp_song_index= music_df[music_df['Track Name']==input_song].index[0]
    
    similaity_scores= cosine_similarity([music_feature_scaled[inp_song_index]], music_feature_scaled)

    similar_song_index= similaity_scores.argsort()[0][::-1][1:num_rec+1]
    
    content_based_recommendations = music_df.iloc[similar_song_index][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [18]:
input_song_name = "Paint The Town Red"
df= pd.DataFrame(content_based_recommendation(input_song_name, 5))
df

,Track Name,Artists,Album Name,Release Date,Popularity
49,Sprinter,"Dave, Central Cee",Sprinter,2023-06-01,92
11,Lovin On Me,Jack Harlow,Lovin On Me,2023-11-10,84
45,You & Me,JENNIE,JENNIE Special Single [You & Me],2023-10-06,83
16,Popular (with Playboi Carti & Madonna) - Music...,"The Weeknd, Playboi Carti, Madonna",Popular (Music from the HBO Original Series),2023-06-02,92
13,Calm Down (with Selena Gomez),"Rema, Selena Gomez",Calm Down (with Selena Gomez),2022-08-25,90


In [40]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendation(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weight_probablity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    input_song_df = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, input_song_df], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [41]:
input_song_name = "Flowers"
recommendations = pd.DataFrame(hybrid_recommendations(input_song_name, num_recommendations=5))
print(recommendations)

                                          Track Name  \
0                                               LALA   
3  Is It Over Now? (Taylor's Version) (From The V...   
4                                 Boy's a liar Pt. 2   
1                                        Single Soon   
2                                        Lovin On Me   

                     Artists               Album Name Release Date  Popularity  
0                Myke Towers           LA VIDA ES UNA   2023-03-23        96.0  
3               Taylor Swift  1989 (Taylor's Version)   2023-10-26        94.0  
4  PinkPantheress, Ice Spice       Boy's a liar Pt. 2   2023-02-03        88.0  
1               Selena Gomez              Single Soon   2023-08-25        87.0  
2                Jack Harlow              Lovin On Me   2023-11-10        84.0  
